In [66]:
import pandas as pd
from sqlalchemy import create_engine
import plotly.express as px

In [2]:
# Provide all credentials  
username =
password = 
host = 
port =
database =

engine = create_engine("mysql+pymysql://" +username+":"+password+"@"+host+":"+str(port)+"/"+database)
query = "select * from superstore_data"
df = pd.read_sql(query,engine)

In [3]:
df

,Row_ID,Order_ID,Order_Date,Ship_Date,Ship_Mode,Customer_ID,Segment,Ship_City,Ship_State,Ship_Postal_Code,Region,Product_ID,Category,Sub_Category,Product_Name,sales,Quantity,Unit_Price,profit
0,1,CA-2022-152156,2022-11-08,2022-11-11,Second Class,CG-12520,Consumer,Mumbai,Maharashtra,400081,Western,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,21837,2,10918,10918
1,2,CA-2022-152156,2022-11-08,2022-11-11,Second Class,CG-12520,Consumer,Bengaluru,Karnataka,560085,Southern,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",61015,3,20338,20338
2,3,CA-2022-138688,2022-06-12,2022-06-16,Second Class,DV-13045,Corporate,Navi Mumbai,Maharashtra,410210,Western,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,1219,2,609,609
3,4,US-2021-108966,2021-10-11,2021-10-18,Standard Class,SO-20335,Consumer,Puducherry,Puducherry,605008,Southern,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,79824,5,15965,15965
4,5,US-2021-108966,2021-10-11,2021-10-18,Standard Class,SO-20335,Consumer,Chennai,Tamil Nadu,600073,Southern,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,1865,2,932,932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9988,9989,CA-2020-110422,2020-01-21,2020-01-23,Second Class,TB-21400,Consumer,New Delhi,Delhi,110084,Northern,FUR-FU-10001889,Furniture,Furnishings,Ultra Door Pull Handle,2105,3,702,702
9989,9990,CA-2023-121258,2023-02-26,2023-03-03,Standard Class,DB-13060,Consumer,Sohna,Haryana,122102,Northern,FUR-FU-10000747,Furniture,Furnishings,Tenex B1-RE Series Chair Mats for Low Pile Car...,7666,2,3833,3833
9990,9991,CA-2023-121258,2023-02-26,2023-03-03,Standard Class,DB-13060,Consumer,Kozhikode,Kerala,673304,Southern,TEC-PH-10003645,Technology,Phones,Aastra 57i VoIP phone,21555,2,10777,10777
9991,9992,CA-2023-121258,2023-02-26,2023-03-03,Standard Class,DB-13060,Consumer,Golaghat,Assam,785621,North Eastern,OFF-PA-10004041,Office Supplies,Paper,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",2467,4,617,617


In [ ]:
df.columns

In [ ]:
df.columns = df.columns.str.lower()

In [ ]:
df

In [144]:
query = """
    select year(order_date) as Year, month(order_date) as Month,
    sum(sales) as Total_Sales
    from superstore_data
    group by 1,2 
    order by 1,2;
"""

monthly_sales = pd.read_sql(query,engine)
fig = px.line(monthly_sales, x="Month", y="Total_Sales",color="Year",title="Monthly Sales Trend By Year")

fig

## Insight

* Sales have shown an increasing trend across the years, with 2023 having the highest overall sales figures compared to previous years.
* There are clear spikes in sales during certain months, especially around March (Month 3), September (Month 9), and November (Month 11), suggesting seasonal demand.
* A significant drop is visible in October (Month 10) across most years, indicating a possible off-season or external  factors impacting sales.
* November and December consistently show high sales across all years, indicating peak seasonality.

In [147]:
customer_segmentation = df.groupby("Customer_ID").agg(Total_Sales = ("sales","sum"),Total_Profit=("profit","sum"),
                                                      Quantity = ("Quantity","sum")).reset_index()
fig= px.scatter(customer_segmentation, x="Total_Sales", y="Total_Profit", size="Quantity", color="Total_Profit",
               title= "Customer Segmentation by Profitability", hover_name="Customer_ID")
fig

# Insight

* The color gradient shows profitability, with yellow representing highly profitable customers and purple representing lower-profit customers.

* A majority of customers have lower sales and profit, while a few outliers with high sales contribute significantly to profit.

In [239]:
Category_data = df.groupby("Category").agg(Total_Sales = ("sales","sum"),Total_Profit = ("profit","sum")).reset_index()
fig = px.bar(Category_data, y= "Category", x=["Total_Sales","Total_Profit"],barmode= "stack", title="Total Sales and Profit by Category",
             orientation="h")

fig.show()

## Insight

* Technology category has the highest total sales (69.7M) and profit (18.5M), making it the most profitable segment.
* Furniture has high sales (61.8M) but relatively lower profit (16M), indicating higher costs or lower margins.
* Office Supplies has moderate sales (59.9M) but still contributes significantly to profit (16.1M).


In [24]:
query = """
    select Segment, sum(sales)*100/(select sum(sales) from superstore_data) as Sales_Distribution
    from superstore_data
    group by 1;
"""

sales_percentage = pd.read_sql(query,engine)

fig = px.pie(sales_percentage, values= "Sales_Distribution", names= "Segment", title = "Sales Distribution by Segment")
fig.show()

## Insight

* Consumer segment dominates sales, contributing 50.6% of the total revenue.
* Corporate segment holds 30.7%, making it the second-largest contributor.
* Home Office segment has the lowest share at 18.7%, indicating potential for growth.

In [192]:
region_data = df.groupby("Region").agg(Total_Sales = ("sales","sum"), Total_Profit = ("profit","sum")).reset_index()

fig_bar = px.bar(region_data, x="Region", y=(["Total_Sales","Total_Profit"]), barmode= "group", 
              title="Total Sales and Profit by Region") 

fig_bar.show()

## Insight

* Southern Region Leads

 Highest total sales (76.2M), significantly higher than other regions.
 Highest profit (20M), making it the most profitable region.
 
* Western Region is the Second Highest

Good sales (39.78M) and profit (10.72M), though lower than the Southern region.

In [237]:
shipMode = df.groupby("Ship_Mode").agg(Total_Sales=("sales","sum"),Total_Profit=("profit","sum")).reset_index()
fig = px.bar(shipMode, x="Ship_Mode", y=["Total_Sales","Total_Profit"], title= "Total Sales and Profit by Ship Mode",
            barmode= "group")
fig

## Insight

 * Standard Class has the highest total sales (113.2M) and profit (29.5M).
   This suggests that most customers prefer this mode due to its affordability and reliability.

 * First Class and Second Class have similar sales levels, with Second Class performing slightly better in sales.
   Profit for both modes is also at a moderate level.

 * Same Day shipping has the lowest sales (10.7M) and profit (2.9M).
   This could be due to high shipping costs, making it a less preferred option for customers.

In [104]:
state = df.groupby("Ship_State").agg(Total_Sales=("sales","sum"),Total_Profit=("profit","sum")).reset_index()

fig= px.scatter(state, x="Total_Sales", y="Total_Profit", hover_name="Ship_State", title= "Total Sales and Profit by State")
fig

## Insight

* Uttar Pradesh, Tamilnadu, Telangana, Karnataka, Maharashtra have significantly higher sales and profit, indicating strong market demand in those regions.
* The highest-performing state Maharashtra has over 30M in sales and more than 8M in profit.
* Many states cluster in the lower-left region, meaning they contribute small sales and profits.

In [105]:
query = """
    select ship_mode, datediff(ship_date, order_date) as Delivery_Time
    from superstore_data;
"""

Delivery = pd.read_sql(query,engine)

fig = px.box(Delivery, x="ship_mode", y="Delivery_Time", title= "Delivery Time by Shipping Mode")
fig

# Insight

1. Delivery Time Distribution:

* Same Day: Almost all deliveries happen on the same day (0 days), with very little variation.
* First Class: Delivery times range from 1 to about 3 days, with a median around 2 days.
* Second Class: Delivery times range from 1 to 5 days, with a median around 3 days.
* Standard Class: Takes the longest, with delivery times ranging from 3 to 7 days, and a median around 5 days.

2. Variability:

* Same Day has the least variation, meaning it is the most consistent.
* Standard Class has the widest range, meaning delivery times can fluctuate significantly.
* First Class and Second Class have moderate variability, with a few outliers.

3. Outliers:

* There are some outliers in First Class and Same Day shipping, which could indicate occasional delays.

In [229]:
category_sales = df.groupby(["Category","Sub_Category"])["sales"].sum().reset_index()
fig = px.treemap(category_sales,path=["Category","Sub_Category"], values="sales", title="Contribution of Category to Total Sales")

fig

## Insight

* Phones (27.5M) and Chairs (27.3M) drive the highest sales, making Technology and Furniture the top categories.
* Office Supplies have broad but lower contributions, with Storage (18.6M) and Binders (16.9M) leading the segment.
* Potential growth areas: Copiers (12.4M), Furnishings (7.6M), and Paper (6.5M) could benefit from targeted promotions.
* Art(2.2M), Supplies (3.8M), Envelops(1.3M) have lower sales

In [235]:
top_products = df.groupby("Product_Name")["sales"].sum().reset_index()
top10_products = top_products.sort_values(by="sales", ascending= False).head(10)
fig = px.bar(top10_products, y="Product_Name", x="sales", orientation="h" ,title="Top 10 Products by Sales")

fig

In [240]:
query = """
    select weekday(order_date) as DayNo, dayname(order_date) as Day, count(Order_ID) as Total_Orders
    from superstore_data
    group by 1,2
    order by 1;
"""

Day = pd.read_sql(query,engine)
fig = px.bar(Day, x="Day", y="Total_Orders", title="Order Volume by Weekday")

fig

## Insight

* Monday sees the highest order volume (1,834 orders), indicating strong demand at the start of the week.
* Wednesday has the lowest order volume (680 orders), suggesting a midweek slowdown.
* Weekends (Saturday & Sunday) experience high order volumes (~1,700+ orders), likely due to increased consumer activity.
* Tuesday and Thursday have moderate order volumes (~1,270–1,295), showing consistency in midweek sales.
* Friday sees a rise in orders (1,464), possibly as people prepare for the weekend.

In [226]:
Data = df.groupby(["Region","Category"])["sales"].sum().reset_index()
fig = px.density_heatmap(Data, x="Region", y="Category", z="sales", title="Sales by Region and Catgeory") #color_continuous_scale = ""

fig

## Insight

* Southern region has the highest sales(22M-28M) across all categories, especially in the Technology and Furniture segments.
* North Eastern region records the lowest sales(<2.48M), indicating a potential market gap or lower demand.
* Western region shows moderate sales(13M-14M) across all categories, suggesting a stable market.
* Office Supplies category sees relatively lower sales compared to Technology and Furniture.
* Technology and Furniture categories perform best in the Southern region, highlighting strong demand.
* Eastern and Central regions have similar mid-range sales(4.5M-8.5M) patterns across all categories.

In [242]:
query = """
    SELECT Category, ROUND((SUM(profit) / SUM(sales))*100,2) AS ProfitMargin
    FROM superstore_data
    GROUP BY 1
    ORDER BY 2 DESC;
"""
profit_margine = pd.read_sql(query,engine)

fig = px.pie(profit_margine, names="Category", values="ProfitMargin", title="Profit Margine by Category")
fig

## Insight

* Office Supplies has the highest profit margin (33.9%), making it the most profitable category.
* Technology (33.4%) and Furniture (32.7%) have nearly equal profit margins, indicating balanced profitability across categories.
* The profit margins are evenly distributed, suggesting that all three categories contribute significantly to overall profitability.